In [1]:
!pip install nltk

import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

nltk.download('stopwords')


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\parva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_excel("finalDataset0.2.xlsx")
data.head()

,teaching,teaching.1,coursecontent,coursecontent.1,examination,Examination,labwork,labwork.1,library_facilities,library_facilities,extracurricular,extracurricular.1
0,0,teacher are punctual but they should also give...,0.0,content of courses are average,1.0,examination pattern is good,-1,"not satisfactory, lab work must include latest...",0.0,library facilities are good but number of book...,1,extracurricular activities are excellent and p...
1,1,Good,-1.0,Not good,1.0,Good,1,Good,-1.0,Not good,1,Good
2,1,Excellent lectures are delivered by teachers a...,1.0,All courses material provide very good knowled...,1.0,Exam pattern is up to the mark and the Cgpa de...,1,Lab work is properly covered in the labs by th...,1.0,Library facilities are excellent in terms of g...,1,Extra curricular activities also help students...
3,1,Good,-1.0,Content of course is perfectly in line with th...,-1.0,Again the university tests students of their a...,1,Good,0.0,Its the best thing i have seen in this univers...,-1,Complete wastage of time. Again this opinion i...
4,1,teachers give us all the information required ...,1.0,content of courses improves my knowledge,1.0,examination pattern is good,1,practical work provides detail knowledge of th...,1.0,library has huge collection of books from diff...,1,extracurricular activities increases mental an...


In [3]:
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r"[^\w\s]", " ", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

In [4]:
text_columns = [
    'teaching', 'coursecontent', 'Examination', 
    'labwork', 'library_facilities', 'extracurricular'
]

for col in text_columns:
    data[col] = data[col].astype(str).apply(preprocess_text)

In [5]:
data['combined_text'] = data[text_columns].apply(
    lambda r: ' '.join(r.values), axis=1
)

In [6]:
X = data['combined_text']
y = data[text_columns]   # multilabel targets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [9]:
model = MultiOutputClassifier(MultinomialNB())
model.fit(X_train_tfidf, y_train)

C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_label.py:302: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  self.y_type_ = type_of_target(y, input_name="y")
C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\multiclass.py:79: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  ys_types = set(type_of_target(x) for x in ys)
C:\Users\parva\AppData\Local\Programs\Python\Python313\

,estimator,MultinomialNB()
,n_jobs,None
,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


In [10]:
y_pred = model.predict(X_test_tfidf)

In [11]:
pred_df = pd.DataFrame(y_pred, columns=y_test.columns)
pred_df.to_excel("predicted_output.xlsx", index=False)

print("Saved predictions to predicted_output.xlsx")

Saved predictions to predicted_output.xlsx


In [12]:
for i, col in enumerate(y_test.columns):
    print(f"\n===== LABEL: {col} =====")
    print(classification_report(y_test[col], y_pred[:, i]))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test[col], y_pred[:, i]))
    print("--------------------------------------")


===== LABEL: teaching =====
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.78      1.00      0.88        29

    accuracy                           0.78        37
   macro avg       0.39      0.50      0.44        37
weighted avg       0.61      0.78      0.69        37

Confusion Matrix:
[[ 0  8]
 [ 0 29]]
--------------------------------------

===== LABEL: coursecontent =====
              precision    recall  f1-score   support

         0 0       0.00      0.00      0.00         2
         1 0       0.95      1.00      0.97        35

    accuracy                           0.95        37
   macro avg       0.47      0.50      0.49        37
weighted avg       0.89      0.95      0.92        37

Confusion Matrix:
[[ 0  2]
 [ 0 35]]
--------------------------------------

===== LABEL: Examination =====
                                                                                                   

C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\parva\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [13]:
import joblib

# Save vectorizer and model
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(model, "sentiment_model.pkl")

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [47]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import joblib

# -------------------- 1️⃣ Training Data --------------------
data = [
    # Positive examples
    {
        "teaching": "The teacher is very helpful",
        "teaching_label": 1,
        "coursecontent": "Content is good",
        "coursecontent_label": 1,
        "Examination": "Exam was easy",
        "Examination_label": 1,
        "labwork": "Labs were useful",
        "labwork_label": 1,
        "library_facilities": "Library is very good",
        "library_facilities_label": 1,
        "extracurricular": "Activities are excellent",
        "extracurricular_label": 1
    },
    # Negative examples
    {
        "teaching": "The teacher is not supportive",
        "teaching_label": -1,
        "coursecontent": "Content is outdated",
        "coursecontent_label": -1,
        "Examination": "Exam was hard",
        "Examination_label": -1,
        "labwork": "Labs were confusing",
        "labwork_label": -1,
        "library_facilities": "Library is inadequate",
        "library_facilities_label": -1,
        "extracurricular": "Activities are boring",
        "extracurricular_label": -1
    },
    # Neutral examples
    {
        "teaching": "The teacher explained the topic",
        "teaching_label": 0,
        "coursecontent": "Content is average",
        "coursecontent_label": 0,
        "Examination": "Exam was standard",
        "Examination_label": 0,
        "labwork": "Labs were okay",
        "labwork_label": 0,
        "library_facilities": "Library has some books",
        "library_facilities_label": 0,
        "extracurricular": "Activities are available",
        "extracurricular_label": 0
    }
]

df = pd.DataFrame(data)

columns = ['teaching', 'coursecontent', 'Examination', 'labwork', 'library_facilities', 'extracurricular']

# -------------------- 2️⃣ Train per-column numeric models --------------------
for col in columns:
    X = df[col]
    y = df[col + '_label']  # -1, 0, 1
    
    model = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english')),
        ('nb', MultinomialNB())
    ])
    
    model.fit(X, y)
    joblib.dump(model, f"{col}_numeric_model.pkl")
    print(f"Trained numeric model for column: {col}")

# -------------------- 3️⃣ Predict numeric sentiment for new feedback --------------------
new_feedback = [
    {
        "teaching": "The teacher is very helpful",       # positive
        "coursecontent": "Content is average",           # neutral
        "Examination": "Exam was easy",                  # positive
        "labwork": "Labs were okay",                     # neutral
        "library_facilities": "Library is very good",   # positive
        "extracurricular": "Activities are available"   # neutral
    },
    {
        "teaching": "The teacher is not supportive",    # negative
        "coursecontent": "Content is outdated",         # negative
        "Examination": "Exam was standard",             # neutral
        "labwork": "Labs were confusing",               # negative
        "library_facilities": "Library has some books", # neutral
        "extracurricular": "Activities are boring"      # negative
    }
]

df_new = pd.DataFrame(new_feedback)
predictions = {}

for col in df_new.columns:
    model = joblib.load(f"{col}_numeric_model.pkl")
    predictions[col] = model.predict(df_new[col])

pred_df = pd.DataFrame(predictions)
print("\nNumeric sentiment predictions (-1 = negative, 0 = neutral, 1 = positive):")
print(pred_df)

Trained numeric model for column: teaching
Trained numeric model for column: coursecontent
Trained numeric model for column: Examination
Trained numeric model for column: labwork
Trained numeric model for column: library_facilities
Trained numeric model for column: extracurricular

Numeric sentiment predictions (-1 = negative, 0 = neutral, 1 = positive):
   teaching  coursecontent  Examination  labwork  library_facilities  \
0         1              0            1        0                   1   
1        -1             -1            0       -1                   0   

   extracurricular  
0                0  
1               -1  
